In [1]:
import os
import json
import pathlib


In [7]:
import os
import json
import pathlib
from datetime import datetime
from dataclasses import dataclass
from typing import Literal

@dataclass
class EmojiConfig:
    folder_path: str
    json_file: str
    sort_by: Literal["time", "name"] = "time"
    prepend: bool = False
    cdn_prefix: str = "https://cdn.jsdelivr.net/gh/infinitesum/Twikoo-emoji@master/Blob/"

def sort_entries(entries: list, sort_by: str) -> list:
    if sort_by == "time":
        # Sort by modification time (newest first)
        entries.sort(key=lambda x: x['_mtime'], reverse=True)
    else:
        # Sort by filename
        entries.sort(key=lambda x: x['text'])
    return entries

def update_blobcat_json(config: EmojiConfig):
    default_data = {
        config.folder_path.split('/')[-1]: {
            "type": "image",
            "container": []
        }
    }
    
    # Load JSON
    if not os.path.exists(config.json_file):
        data = default_data
    else:
        try:
            with open(config.json_file, 'r') as f:
                data = json.load(f)
                if config.folder_path.split('/')[-1] not in data or 'container' not in data[config.folder_path.split('/')[-1]]:
                    data = default_data
        except json.JSONDecodeError:
            data = default_data
    
    existing_emojis = {item['text'] for item in data[config.folder_path.split('/')[-1]]['container'] 
                      if isinstance(item, dict) and 'text' in item}
    
    # Get files
    new_entries = []
    if os.path.exists(config.folder_path):
        image_files = []
        for pattern in ['*.png', '*.gif', '*.jpg', '*.jpeg', '*.webp']:
            image_files.extend(list(pathlib.Path(config.folder_path).glob(pattern)))
            image_files.extend(list(pathlib.Path(config.folder_path).glob(pattern.upper())))
        
        for file in image_files:
            name = file.stem.lower()
            if 'blobcat' in name:
                name = name.replace('blobcat', '').replace('ablob', '')
                
            if name not in existing_emojis:
                entry = {
                    "text": name,
                    "icon": f"<img src='{config.cdn_prefix}{file.name}'>",
                    "_mtime": os.path.getmtime(file)
                }
                new_entries.append(entry)
        
        if new_entries:
            # Sort entries
            new_entries = sort_entries(new_entries, config.sort_by)
            # Remove temporary _mtime field
            for entry in new_entries:
                del entry['_mtime']
                
            if config.prepend:
                data[config.folder_path.split('/')[-1]]['container'] = new_entries + data[config.folder_path.split('/')[-1]]['container']
                position = "beginning"
            else:
                data[config.folder_path.split('/')[-1]]['container'].extend(new_entries)
                position = "end"
            
            with open(config.json_file, 'w') as f:
                json.dump(data, f, indent=4)
            print(f"Added {len(new_entries)} new emojis at the {position} (sorted by {config.sort_by})")
        else:
            print("No new emojis found")
    else:
        print(f"Folder {config.folder_path} not found")





In [8]:
# Usage examples
config = EmojiConfig(
    folder_path="./nobeko",
    json_file="nobeko.json",
    sort_by="time",  # "time" or "name", default: recent first
    prepend=True,    # True: add to beginning, False: add to end
    cdn_prefix="https://cdn.jsdelivr.net/gh/infinitesum/Twikoo-emoji@master/nobeko/"  # Custom CDN prefix
)

In [9]:
update_blobcat_json(config)

Added 44 new emojis at the beginning (sorted by time)
